In [1]:
import torch

print("CUDA 사용 가능:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))
    print("CUDA 버전:", torch.version.cuda)
    print("PyTorch 버전:", torch.__version__)
else:
    print("GPU가 인식되지 않았습니다.")

CUDA 사용 가능: True
GPU 이름: NVIDIA GeForce RTX 4070 SUPER
CUDA 버전: 11.8
PyTorch 버전: 2.5.1+cu118


In [2]:
# 허깅페이스 로그인
from huggingface_hub import login

login()

In [ ]:
# 한 줄씩 실행
# pip uninstall tensorflow -y
# pip install --upgrade --force-reinstall transformers

In [15]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# # 9/5B
# model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# # model_name = "beomi/KoAlpaca-Polyglot-5.8B"


# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",  # GPU 자동 배분
#     torch_dtype="auto"
# )

# llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# CPU로 모델 로드 (device_map 제거)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto"  # 또는 명시적으로 torch.float16 등
)

# PAD 토큰 추가 및 토큰 임베딩 사이즈 조정
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# 모델을 GPU로 이동 (예: cuda 사용)
model = model.to("cuda")

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [8]:
### prac
prompt = """
다음은 연예인 스캔들과 관련된 기사입니다:

"2020년 5월 17일 발매된 슈가의 믹스테잎 수록곡이 특정 단체와 관련된 인물의 연설을 사용하여 논란이 되었습니다."

이 사건의 주요 키워드는 무엇인가요? 핵심 단어 3~5개 정도를 중심 문장으로 설명해주세요.
"""

output = llm(prompt, max_new_tokens=150, temperature=0.7, 
             do_sample=True  # 샘플링 활성화
             )[0]['generated_text']
print(output)



# response = llm(prompt, max_new_tokens=150, temperature=0.7, do_sample=True)[0]['generated_text']
# print(response)

# from keybert import KeyBERT

# kw_model = KeyBERT()

# keywords = kw_model.extract_keywords(response, top_n=5, stop_words='korean')

# print("▶ 추출된 키워드:")
# for kw in keywords:
#     print("-", kw[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



다음은 연예인 스캔들과 관련된 기사입니다:

"2020년 5월 17일 발매된 슈가의 믹스테잎 수록곡이 특정 단체와 관련된 인물의 연설을 사용하여 논란이 되었습니다."

이 사건의 주요 키워드는 무엇인가요? 핵심 단어 3~5개 정도를 중심 문장으로 설명해주세요.


"2020년 5월 17일 발매된 슈가의 믹스테잎 수록곡이 특정 단체와 관련된 인물의 연설을 사용하여 논란이 되었습니다."

중요 키워드: wumixsteper, 2020년 5월 17일, 슈가, 믹스테잎 수록곡, 특정 단체와 관련된 인물의 연설,


In [16]:
### prac2
prompt = """
다음 기사 내용을 분석하여, 
1. 기사에서 다루는 스캔들의 핵심 키워드를 중복 없이 3~5개 추출해 주세요. 스캔들에 포함된 사건과 관련된 요소가 있다면 반드시 포함해 주세요.
2. 해당 스캔들의 유형을 분류할 때, 스캔들이 어떤 사건이나 논란(예: 음원 샘플링 논란, 가사/표현 논란, 사회적/정치적 이슈 등)에 근거하여 발생했는지를 명확하게 설명해 주세요. 특히, 인민사원 집단 자살 사건과 연결된 부분은 반드시 언급하여 분류에 반영해 주세요.

출력 형식은 반드시 아래의 두 부분만 작성해 주세요:
- 핵심 키워드: [키워드1, 키워드2, ...]
- 스캔들 유형: [분류 결과]


아래 기사를 참고하여 작업해 주세요.
--------------------------------------------------
2020년 5월 17일 발매된 슈가의 믹스테잎
D-2
의 수록곡 <어떻게 생각해?>의 도입부에
인민사원 집단 자살 사건
의 주범으로 악명높은
짐 존스
의 육성 연설이 샘플링되어 삽입되어 있다는 게 밝혀져 논란이 되었다. 이에 대해 슈가가 악플러들에 대한 반어적 비유를 사용하기 위해 짐 존스의 연설을 삽입한 거 아니냐는 의견과 존스타운과 전혀 관계없는 노래에 굳이 범죄자의 육성을 삽입한 이유가 궁금하다는 의견 등등 갑론을박이 벌어졌다.
인민사원 집단 자살 사건
이 국내에서는 별로 알려진 사건이 아니지만, 해당 사건이 일어난 국가 내에서는 꾸준히 교육시킬 정도로 심각했던 사건인 것은 사실이다.
곡에 담긴 연설의 부분은 "당신은 죽더라도 살 것이다. 살아서 믿는 자는 결코 죽지 않을 것이다."라는 내용이다. 다만 원래 이 문장은 요한복음 11장 25절에 있는 구절이다. 즉, 사이비 목사가 성경 내용을 악의적으로 이용한 것이다. 하지만 해당 연설 내용은 집단 자살 사건을 의미하기 때문에 문제가 된다. 해외에서도 짐 존스를 소재로 삼은 곡들이 꽤 있다. 포스트 말론의 '존스타운'과 릴 우지 버트의 'Leaders'는 가사에서 짐 존스가 언급되며,
BROCKHAMPTON
의 '1998 TRUMAN'는 짐 존스의 음성이 들어간다. 그러나 이 곡들은 모두 맥락의 부합성 면에서 슈가의 노래와 큰 차이점을 보인다.
'존스타운'의 경우는 2분이 채 되지 않는 짧은 곡으로 앞 트랙의 흐름을 이어가는 역할이고, 'Leaders'의 경우에는 '짐 존스처럼 교단을 이끌어'라며 자신의 인기를 과시하기 위한 비유로 써 먹었다. '1998 TRUMAN'은 거대한 환상같은 현실 속 방탕한 삶을 사는 청년들의 현실을 폭로하는 내용으로, '너흰 자유가 아냐, 너흰 노예야!'라 외치는 짐 존스의 육성이 곡의 테마를 잡는다. 해당 아티스트들이 곡에서 짐 존스를 언급한 의도는 각자 다르고, 이에 대한 평가도 '범죄자 미화다', '예술의 영역이다', '수많은 피해자를 낳은 범죄자인데 육성 삽입 자체가 적절치 못했다' 등등 다 달랐다. 하여튼 슈가의 노래와 달리 짐 존스가 왜 등장했는지 그 의도와 맥락의 부합성이 분명하다는 공통점이 있다.
해명
논란이 일자 빅히트 엔터테인먼트(현
빅히트 뮤직
)는 "도입부 연설 보컬 샘플은 해당 곡의 트랙을 작업한 프로듀서가 특별한 의도 없이 연설자를 알지 못한 상태에서 곡 전체의 분위기를 고려해 선정했다. 해당 연설 보컬 샘플을 선정한 이후 회사는 내부 프로세스에 따라 내용의 적정성을 확인하는 절차를 진행했다. 하지만 선정 및 검수 과정에서 내용상 부적절한 샘플임을 인지하지 못하고 곡에 포함하는 오류가 있었다."며 해명하고 공식 사과했다.
#
이후 연설 음성을 삭제한 음원을 재발매하였다. 이후 같은 곡에서 베트남 전범 범죄자의 연설을 사용했다는 루머가 추가로 나돌기도 하였으나 소속사인 빅히트 엔터테인먼트 측에서 "사실무근"이라며 이를 일축했다. 해당 소문은 단순 허위사실 루머인 것으로 확인되었다.
#
확인 결과, 논란이 된 짐 존스의 연설 샘플은 빅히트 프로듀서가 직접 짐 존스 연설 원본에서 샘플링한 것이 아니라 스플라이스라는 샘플 플랫폼에서 판매되고 있는 샘플이었다.
#
해당 곡의 트랙을 작업한 빅히트 프로듀서는 연설자가 누군지 모르고 실수로 해당 연설 보컬 샘플을 선정한 것으로 추측된다.
또한 짐 존스 연설 샘플이 사용된 트랙 <어떻게 생각해?>의 크레딧을 확인해보면 슈가가 프로듀서 명단에 들어가있는 다른 곡들과 다르게 해당 곡은
EL CAPITXN
, GHSTLOOP만 프로듀서 명단에 들어가 있는 것을 확인할 수 있다. 즉, 실수로 해당 연설 보컬 샘플을 선정한 프로듀서는 슈가가 아니며 슈가는 이 노래에는 트랙이 아니라 작사, 녹음 등의 작업에만 참여했다.
2.
군대 관련 가사 논란
[편집]
국회
에서까지 언급된 사건으로 짐 존스 연설 샘플링 논란이 되었던 곡 <어떻게 생각해?>에 "Woo Woo, 군대는 때되면 알아서들 갈 테니까 우리 이름 팔아먹으면서 숟가락을 얹으려고 한 새끼들 싸그리 다 닥치길"이라는 가사가 대중예술인 예술체육요원 대체복무 찬반 논란 때문에 화제가 되었고,
국정감사
에까지 이 가사가 오르내리게 되었다. 이후 슈가가 BTS에서 혼자만
사회복무요원
으로 복무하게 되어 한 번 더 논란이 되었다.
#
[[https://www.gynews.kr/news/articleView.html?idxno=20891|#]
3.
코로나바이러스감염증-19 관련 실언 논란
[편집]
--------------------------------------------------
"""

output = llm(prompt, max_new_tokens=150, temperature=0.7, 
             do_sample=True  # 샘플링 활성화
             )[0]['generated_text']
print(output)


다음 기사 내용을 분석하여, 
1. 기사에서 다루는 스캔들의 핵심 키워드를 중복 없이 3~5개 추출해 주세요. 스캔들에 포함된 사건과 관련된 요소가 있다면 반드시 포함해 주세요.
2. 해당 스캔들의 유형을 분류할 때, 스캔들이 어떤 사건이나 논란(예: 음원 샘플링 논란, 가사/표현 논란, 사회적/정치적 이슈 등)에 근거하여 발생했는지를 명확하게 설명해 주세요. 특히, 인민사원 집단 자살 사건과 연결된 부분은 반드시 언급하여 분류에 반영해 주세요.

출력 형식은 반드시 아래의 두 부분만 작성해 주세요:
- 핵심 키워드: [키워드1, 키워드2, ...]
- 스캔들 유형: [분류 결과]


아래 기사를 참고하여 작업해 주세요.
--------------------------------------------------
2020년 5월 17일 발매된 슈가의 믹스테잎
D-2
의 수록곡 <어떻게 생각해?>의 도입부에
인민사원 집단 자살 사건
의 주범으로 악명높은
짐 존스
의 육성 연설이 샘플링되어 삽입되어 있다는 게 밝혀져 논란이 되었다. 이에 대해 슈가가 악플러들에 대한 반어적 비유를 사용하기 위해 짐 존스의 연설을 삽입한 거 아니냐는 의견과 존스타운과 전혀 관계없는 노래에 굳이 범죄자의 육성을 삽입한 이유가 궁금하다는 의견 등등 갑론을박이 벌어졌다.
인민사원 집단 자살 사건
이 국내에서는 별로 알려진 사건이 아니지만, 해당 사건이 일어난 국가 내에서는 꾸준히 교육시킬 정도로 심각했던 사건인 것은 사실이다.
곡에 담긴 연설의 부분은 "당신은 죽더라도 살 것이다. 살아서 믿는 자는 결코 죽지 않을 것이다."라는 내용이다. 다만 원래 이 문장은 요한복음 11장 25절에 있는 구절이다. 즉, 사이비 목사가 성경 내용을 악의적으로 이용한 것이다. 하지만 해당 연설 내용은 집단 자살 사건을 의미하기 때문에 문제가 된다. 해외에서도 짐 존스를 소재로 삼은 곡들이 꽤 있다. 포스트 말론의 '존스타운'과 릴 우지 버트의 'Leaders'는 가사에서 짐 존스가 언급되며,
BROCKHA

# 모델 저장

In [20]:
save_path = "./mistral-7b-local"

# 토크나이저 저장
tokenizer.save_pretrained(save_path)

# 모델 저장
model.save_pretrained(save_path)

c:\Users\wjdrl\anaconda3\envs\jsy\Lib\site-packages\transformers\modeling_utils.py:3405: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(


Saving checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# 모델 불러오기

#### 크롤링한 파일 ####

In [3]:
# 허깅페이스 로그인
from huggingface_hub import login

login()

In [25]:
df = pd.read_csv('./data/YG논란_크롤링.csv')
df.columns

Index(['소속사', '그룹', '연예인 이름', '사건 날짜', '사건 내용 (날짜+내용 전체)'], dtype='object')

In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from tqdm import tqdm

# 컬럼명 매핑
COLUMN_NAME_MAP = {
    "소속사": "agency",
    "그룹": "group",
    "연예인 이름": "name",
    "사건 날짜": "date",
    "사건 내용 (날짜+내용 전체)": "content"
}

def load_model(model_name="mistralai/Mistral-7B-Instruct-v0.1"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype="auto"
    )
    llm = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return llm

def extract_keywords_and_label(llm, text):
    prompt = f"""
다음은 연예인 논란과 관련된 기사입니다. 먼저 핵심 키워드 3~5개를 추출하고,
이 키워드들을 기반으로 논란 유형을 하나로 분류해주세요.

기사:
\"{text}\"

출력 형식:
- 키워드1
- 키워드2
- ...
- 논란 유형: [분류 결과만 간단하게]
"""
    output = llm(prompt, max_new_tokens=200, temperature=0.7, do_sample=True)[0]['generated_text']
    
    lines = output.strip().splitlines()
    keywords = [line.strip('-• ').strip() for line in lines if line.strip().startswith(('-', '•')) and '논란 유형' not in line]
    label_line = [line for line in lines if '논란 유형' in line]
    label = label_line[0].split(":", 1)[-1].strip() if label_line else "미확인"

    return keywords, label

def load_and_classify(csv_path, article_col="사건 내용 (날짜+내용 전체)", llm=None):
    # CSV 파일 읽기
    df = pd.read_csv(csv_path)
    
    # 컬럼명 공백 제거 (앞뒤 공백 제거)
    df.columns = df.columns.str.strip()
    
    # 결측치 제거: article_col 기준
    df = df[df[article_col].notna()].copy()
    
    # 컬럼명 영어로 변경
    df.rename(columns=COLUMN_NAME_MAP, inplace=True)
    
    keywords_list = []
    labels_list = []
    
    # 분류 작업: 기사 내용은 'content' 컬럼이 됨
    for article in tqdm(df["content"], desc="분류 중"):
        try:
            keywords, label = extract_keywords_and_label(llm, article)
        except Exception as e:
            keywords, label = [], "오류"
        keywords_list.append(keywords)
        labels_list.append(label)
    
    df["keywords"] = keywords_list
    df["category"] = labels_list
    
    return df

def classify_multiple_csvs(file_list, article_col, llm):
    df_list = []
    for file_path in file_list:
        print(f"\n📄 처리 중: {file_path}")
        df = load_and_classify(file_path, article_col=article_col, llm=llm)
        df_list.append(df)
    return df_list

In [5]:
# 처리할 CSV 파일 목록
csv_paths = ["./data/HYBE논란_크롤링.csv", "./data/YG논란_크롤링.csv"]

# LLM 모델 로드
llm = load_model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [ ]:
# 각 CSV 파일에 대해 분류 작업 진행
dfs = classify_multiple_csvs(file_list=csv_paths, article_col="사건 내용 (날짜+내용 전체)", llm=llm)

# 결과를 각각의 CSV 파일로 저장
for csv_path, df in zip(csv_paths, dfs):
    base_name = csv_path.split("/")[-1].split(".")[0]
    output_path = f"./data/{base_name}_분류결과.csv"
    df.to_csv(output_path, index=False)
    print(f"분류 결과 저장됨: {output_path}")


c:\Users\wjdrl\anaconda3\envs\jsy\Lib\site-packages\accelerate\utils\modeling.py:1569: UserWarning: Current model requires 128 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.
Device set to use cpu



📄 처리 중: ./data/HYBE논란_크롤링.csv


분류 중: 100%|██████████| 76/76 [5:11:21<00:00, 245.81s/it]



📄 처리 중: ./data/YG논란_크롤링.csv


분류 중: 100%|██████████| 44/44 [2:52:46<00:00, 235.60s/it]

분류 결과 저장됨: ./data/HYBE논란_크롤링_분류결과.csv
분류 결과 저장됨: ./data/YG논란_크롤링_분류결과.csv


In [8]:
df = pd.read_csv("./data/HYBE논란_크롤링.csv")
for col in df.columns:
    print(repr(col))

'소속사'
'그룹'
'연예인 이름'
'사건 날짜'
'사건 내용 (날짜+내용 전체)'
